NOTE: This is a Jupyter Notebook, so it won't be used in production, only for testing

Resources:
- https://quantum-computing.ibm.com/composer/files/new
- https://wybiral.github.io/quantum/
- https://jarrodmcclean.com/basic-quantum-circuit-simulation-in-python/
- https://quantumcomputing.stackexchange.com/questions/29454/why-do-quantum-computing-simulators-have-the-measurement-function

# Quantum Computation

In [747]:
import numpy as np
import scipy as sp

In [748]:
### basic stuff

# zero ket
ZKet = np.array([1,0])
# one ket
OKet = np.array([0,1])

### operator matrices

# identity
Id = np.array([
    [1,0],
    [0,1]
])
# pauli x
X = np.array([
    [0,1],
    [1,0]
])
# pauli y
Y = np.array([
    [0,0-1j],
    [0+1j,0]
])
# pauli z
Z = np.array([
    [1,0],
    [0,-1]
])
# hadamard
H = 1/np.sqrt(2) * np.array([
    [1,1],
    [1,-1]
])
# phase
S = np.array([
    [1,0],
    [0,0+1j]
])
# pi over eight
T = np.array([
    [1,0],
    [0,np.exp((0+1j)*np.pi/4)]
])
# |0><0>
P_0 = np.array([
    [1,0],
    [0,0]
])
# |1><1>
P_1 = np.array([
    [0,0],
    [0,1]
])

In [749]:
class QState:
    def __init__(self, qbit_cnt, info=False):
        self.qbit_cnt = qbit_cnt
        # create state |00...0> with qbit_cnt amount of zeros
        self.state = np.zeros(2**self.qbit_cnt, dtype=complex)
        self.state[0] = 1

        if info:
            max_bin_width = len(bin(len(self.state) - 1)[2:])
            binary_str = bin(0)[2:].zfill(max_bin_width)
            print("Initial state: |{}>".format(binary_str))
    
    # show all states including the impossible one (unless
    # otherwise noted) and show according probabilities
    # DISCLAIMER: rounding/floating point errors may occur
    def show_state_and_probs(self, reduced=False):
        max_len = 0
        for element in self.state:
            if reduced and element == 0+0j:
                continue

            curr_len = max(len(str(element.real)), len(str(element.imag)))
            if curr_len > max_len:
                max_len = curr_len
        
        max_bin_width = len(bin(len(self.state) - 1)[2:])

        for i, element in enumerate(self.state):
            if reduced and element == 0+0j:
                continue

            real_part_str = "{:.{width}f}".format(element.real, width=max_len)
            imaginary_part_str = "{:.{width}f}".format(element.imag, width=max_len)
            
            formatted_element = ("{}+{}i".format(real_part_str, imaginary_part_str) if element.imag >= 0 else
                                 "{}{}i".format(real_part_str, imaginary_part_str))

            binary_str = bin(i)[2:].zfill(max_bin_width)

            prob_str = str(round((np.absolute(element)**2)*100, 4))

            if i > 0 and element.real >= 0:
                print("+{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            elif i > 0 and element.real < 0:
                print("{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            elif i <= 0 and element.real >= 0:    
                print(" {}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))
            else:
                print("{}|{}>\t-> {}%".format(formatted_element, binary_str, prob_str))

    ###
    ### GATE CREATION HELPER ###
    ###
    
    # e.g. turns [A, B, C, D] into A ⊗ B ⊗ C ⊗ D
    def op_mats_arr_to_tens(self, op_mats_arr):
        result = np.array([[1.0+0j]], dtype=complex)
        for op_mat in op_mats_arr:
            result = np.kron(result, op_mat)
        
        return result

    def sing_qbit_op(self, pos, op_mat):
        op_mats_arr = (
              (pos)*[Id]
            + (1)*[op_mat]
            + (self.qbit_cnt-pos-1)*[Id]
        )
        gate = self.op_mats_arr_to_tens(op_mats_arr)
        self.state = np.matmul(gate, self.state)
    
    # e.g. turns np.array([a, b, c, d])
    # into [[a, ZKet, ZKet], [b, ZKet, OKet], [c, OKet, ZKet], [d, OKet, OKet]]
    # both represent a*ZKET⊗ZKet + b*ZKet⊗OKet + c*OKet⊗ZKet + d*OKet⊗OKet
    def decomp_state(self, comp):
        decomp = []

        max_bin_width = len(bin(len(comp) - 1)[2:])
        for i in range(len(comp)):
            binary_str = bin(i)[2:].zfill(max_bin_width)
            
            sub_decomp = [comp[i]]
            for letter in binary_str:
                if letter == "0":
                    sub_decomp.append(ZKet)
                elif letter == "1":
                    sub_decomp.append(OKet)
            decomp.append(sub_decomp)
        
        return decomp
    
    # e.g. turns [[a, ZKet, ZKet], [b, ZKet, OKet], [c, OKet, ZKet], [d, OKet, OKet]]
    # into np.array([a, b, c, d])
    # both represent a*ZKET⊗ZKet + b*ZKet⊗OKet + c*OKet⊗ZKet + d*OKet⊗OKet
    def comp_state(self, decomp):
        # print(len(decomp[0]))
        state = np.zeros(2**(len(decomp[0])-1), dtype=complex)

        # print("comp_state: decomp", decomp)
        for element in decomp:
            #print("comp_state: element", element)
            #print("comp_state: first part of it", self.op_mats_arr_to_tens(element[1:])[0])
            #print("comp_state: what we add to state", element[0]*self.op_mats_arr_to_tens(element[1:])[0])
            # print("comp_state: element {} as part of {}".format(element[1:], decomp))
            state += element[0]*self.op_mats_arr_to_tens(element[1:])[0]

        return state

    def ctrl(self, ctrl_pos, targ_func, *targ_pos_args):
        decomp = self.decomp_state(self.state)
        print("ctrl: full decomp\t", decomp)

        print("----")
        for i in range(len(decomp)):
            print("ctrl: part of full decomp in loop:", decomp[i])
            for j in range(1, len(decomp[i])):
                if j-1 != ctrl_pos or np.array_equal(decomp[i][j], ZKet):
                    continue
                print("ctrl: yes")

                orig_qbit_cnt = self.qbit_cnt
                self.qbit_cnt = len(decomp[i])-1

                orig_state = self.state
                self.state = self.comp_state([decomp[i]])
                print("part0 self.state", self.state)
                targ_func(*targ_pos_args)
                print("part1 self.state", self.state)
                print("part2 decomp of self.state", self.decomp_state(self.state))
                # decomp[i] = self.decomp_state(self.state) ##################
                decomp.pop(i)
                decomp += self.decomp_state(self.state)
                
                self.qbit_cnt = orig_qbit_cnt
                self.state = orig_state
                break # out of second for loop

            print("updataeeee:\t\t", decomp)
            print("--")

        print("crazyyy")
        self.state = self.comp_state(decomp)
        print("insane")

    def ctrl_ctrl(self, ctrl_pos1, ctrl_pos2, targ_func, *targ_pos_args):
        pass
     
    ###
    ### SINGLE-QUBIT GATES ###
    ###

    def pauli_x(self, pos):
        print("POSITION IN PAULI_X:", pos)
        self.sing_qbit_op(pos, X)

    def pauli_y(self, pos):
        self.sing_qbit_op(pos, Y)

    def pauli_z(self, pos):
        self.sing_qbit_op(pos, Z)

    def hadamard(self, pos):
        self.sing_qbit_op(pos, H)
    
    def phase(self, pos):
        self.sing_qbit_op(pos, S)

    def pi_ov_8(self, pos):
        self.sing_qbit_op(pos, T)

    ###
    ### MULTI-QUBIT GATES ###
    ###

    # doesn't matter whether ctrl_pos smaller or bigger than targ_pos
    def cnot(self, ctrl_pos, targ_pos):
        self.ctrl(ctrl_pos, self.pauli_x, targ_pos)

    def cz(self, ctrl_pos, targ_pos):
        self.ctrl(ctrl_pos, self.pauli_z, targ_pos)
    
    def cp(self, ctrl_pos, targ_pos):
        self.ctrl(ctrl_pos, self.phase, targ_pos)
    
    def toffoli(self, ctrl_pos1, ctrl_pos2, targ_pos):
        self.ctrl_ctrl(ctrl_pos1, ctrl_pos2, self.pauli_x, targ_pos)

    def swap(self, pos1, pos2):
        # using special decomposition to achieve it's functionality:
        self.cnot(pos1, pos2)
        self.cnot(pos2, pos1)
        self.cnot(pos1, pos2)

    def fredkin(self, ctrl_pos, swap_pos1, swap_pos2):
        self.ctrl(ctrl_pos, self.swap, swap_pos1, swap_pos2)



In [750]:
# ### Testing

# qs = QState(4, True)

# qs.pauli_x(1)
# qs.pauli_x(2)
# qs.hadamard(0)
# qs.hadamard(3)
# qs.cnot(0,2)
# # qs.swap(1,3)

# qs.show_state_and_probs(True)

In [751]:
# qs = QState(2, True)

# qs.hadamard(0)
# qs.pauli_x(1)
# qs.cz(0,1)

# qs.show_state_and_probs(True)

In [752]:
# a = [55,33,22]

# b = a[1:]
# print(b)

In [753]:
a = [44,333,5555]

print(a)
a.append(0)
print(a)

[44, 333, 5555]
[44, 333, 5555, 0]


In [754]:
# qs = QState(2, True)

# qs.hadamard(0)
# qs.show_state_and_probs(True)

# a = qs.decomp_state(qs.state)
# print("a1 decomp:", a)
# print("a1 comp:", qs.comp_state(a))
# a.append([5+0j, np.array([1,0]), np.array([0,1])])
# print("a2 decomp:", a)
# print("a2 comp:", qs.comp_state(a))

In [755]:
# ################# MAIN TEST ###################

qs = QState(2, True)

qs.hadamard(0)
qs.cnot(0,1)

qs.show_state_and_probs(True)

Initial state: |00>
ctrl: full decomp	 [[(0.7071067811865475+0j), array([1, 0]), array([1, 0])], [0j, array([1, 0]), array([0, 1])], [(0.7071067811865475+0j), array([0, 1]), array([1, 0])], [0j, array([0, 1]), array([0, 1])]]
----
ctrl: part of full decomp in loop: [(0.7071067811865475+0j), array([1, 0]), array([1, 0])]
updataeeee:		 [[(0.7071067811865475+0j), array([1, 0]), array([1, 0])], [0j, array([1, 0]), array([0, 1])], [(0.7071067811865475+0j), array([0, 1]), array([1, 0])], [0j, array([0, 1]), array([0, 1])]]
--
ctrl: part of full decomp in loop: [0j, array([1, 0]), array([0, 1])]
updataeeee:		 [[(0.7071067811865475+0j), array([1, 0]), array([1, 0])], [0j, array([1, 0]), array([0, 1])], [(0.7071067811865475+0j), array([0, 1]), array([1, 0])], [0j, array([0, 1]), array([0, 1])]]
--
ctrl: part of full decomp in loop: [(0.7071067811865475+0j), array([0, 1]), array([1, 0])]
ctrl: yes
part0 self.state [0.        +0.j 0.        +0.j 0.70710678+0.j 0.        +0.j]
POSITION IN PAULI_X:

In [756]:
# def foo(a, b):
#     return a+b

# def bar(*args):
#     return foo(*args)

# print(bar(3,5))

In [757]:
for i in range(5):
    for j in range(5):
        if j == 2:
            break
        print(i, j)

0 0
0 1
1 0
1 1
2 0
2 1
3 0
3 1
4 0
4 1


In [758]:
# comp = np.array([11,22,33,44])

# max_bin_width = len(bin(len(comp) - 1)[2:])

# for i in range(len(comp)):
#     binary_str = bin(i)[2:].zfill(max_bin_width)
#     print(binary_str)

In [759]:
# def decomp_state(comp):
#     decomp = []

#     max_bin_width = len(bin(len(comp) - 1)[2:])
#     for i in range(len(comp)):
#         binary_str = bin(i)[2:].zfill(max_bin_width)
        
#         sub_decomp = [comp[i]]
#         for letter in binary_str:
#             if letter == "0":
#                 sub_decomp.append(ZKet)
#             elif letter == "1":
#                 sub_decomp.append(OKet)
#         decomp.append(sub_decomp)
    
#     return decomp

# a = decomp_state(np.array([11,22,33,44]))

# def comp_state(decomp):
#     state = []

#     for element in decomp:
#         state.append(element[0])

#     return state

# print(a)
# print(comp_state(a))

# Protocol

In [760]:
input_info = {"qbit_cnt": 3}
input_gates = [{"type": "pauli_x", "qbit": [0]}, {"type": "cnot", "qbit": [0, 1]}]
# careful that in [0, 1] the first number MUST be the control qubit, and the other one the target qubit
# SO EVERYTHING MUST BE STRUCTURED ACCORDING TO THE GATE FUNCTIONS (that are listed in QState) 

qs = QState(input_info["qbit_cnt"], True)

for input_gate in input_gates:
    gate_func = getattr(qs, input_gate["type"])
    gate_func(*input_gate["qbit"])

qs.show_state_and_probs(True)

Initial state: |000>
POSITION IN PAULI_X: 0
ctrl: full decomp	 [[0j, array([1, 0]), array([1, 0]), array([1, 0])], [0j, array([1, 0]), array([1, 0]), array([0, 1])], [0j, array([1, 0]), array([0, 1]), array([1, 0])], [0j, array([1, 0]), array([0, 1]), array([0, 1])], [(1+0j), array([0, 1]), array([1, 0]), array([1, 0])], [0j, array([0, 1]), array([1, 0]), array([0, 1])], [0j, array([0, 1]), array([0, 1]), array([1, 0])], [0j, array([0, 1]), array([0, 1]), array([0, 1])]]
----
ctrl: part of full decomp in loop: [0j, array([1, 0]), array([1, 0]), array([1, 0])]
updataeeee:		 [[0j, array([1, 0]), array([1, 0]), array([1, 0])], [0j, array([1, 0]), array([1, 0]), array([0, 1])], [0j, array([1, 0]), array([0, 1]), array([1, 0])], [0j, array([1, 0]), array([0, 1]), array([0, 1])], [(1+0j), array([0, 1]), array([1, 0]), array([1, 0])], [0j, array([0, 1]), array([1, 0]), array([0, 1])], [0j, array([0, 1]), array([0, 1]), array([1, 0])], [0j, array([0, 1]), array([0, 1]), array([0, 1])]]
--
ctrl